In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation  # Import FuncAnimation
import numpy as np
import csv
import json
import csv
import numpy as np



def generate_rmm_array(shmm_data):
    num_tasks = len(shmm_data)

    position_values = [pose[0] for pose in shmm_data]
    time_values = [pose[1] for pose in shmm_data]
    mission_tim_values = [pose[2] for pose in shmm_data]

    # Increase noise standard deviation
    position_noise = np.random.normal(0, 1, num_tasks)  # Adjusted standard deviation
    time_noise = np.random.normal(0, 3, num_tasks)

    rmm_array = []
    prev_time = 0  # Variable to track the previous time value

    for i in range(num_tasks):
        position_value1 = int(np.round(np.abs(position_values[i][0] + position_noise[i])))
        position_value2 = int(np.round(np.abs(position_values[i][1] + np.random.normal(0, 0.1))))
        time_taken = round((np.abs(time_values[i] + time_noise[i])),2)
        mission_tim = int(mission_tim_values[i])

        # Clip position values to be within specified limits
        position_value1 = np.clip(position_value1, 0, 5)
        position_value2 = np.clip(position_value2, 0, 3)

        rmm_array.append([i, position_value1, position_value2, time_taken, mission_tim])

        # Update the previous time value
        prev_time = time_taken + 1  # Increase time for the next iteration

    return rmm_array






# Function to calculate L1 norm between two arrays
def calculate_l1_norm(array1, array2):
    norm = (np.array(array2) - np.array(array1))
    return norm


def bayesian_probabilistic_update_general(original_value, deviation, threshold, uncertainty_factor):
    # Expectation step: compute the expected value of the latent variable
    expected_value = original_value + deviation

    # Maximization step: update the parameter using the expected value
    updated_value = expected_value

    # Update only if the deviation crosses the threshold
    updated_value = updated_value if abs(deviation) > threshold else original_value
    return updated_value










# Function to generate delay message based on scenario
def generate_delay_message(robot_number, X, Y, A, B, scenario=None):
    if scenario == "emergency_stop":
        message_template = (
            f"Commander, Robot {robot_number} has encountered an emergency situation "
            f"and has stopped abruptly at position {A}. Further assessment is required.\n"
            f"**************************************************\n"
        )
    elif scenario == "obstacle_detected":
        message_template = (
            f"Commander, Robot {robot_number} has detected an obstacle in its path and is navigating around it. "
            f"Current position: {A}.\n"
            f"**************************************************\n"
        )
    elif scenario == "low_battery":
        message_template = (
            f"Commander, Robot {robot_number}'s battery level is critically low. "
            f"It will pause its mission and return to base for recharging. Current position: {A}.\n"
            f"**************************************************\n"
        )
    elif scenario == "communication_failure":
        message_template = (
            f"Commander, Communication with Robot {robot_number} has been lost. "
            f"Please investigate the issue. Last known position: {A}.\n"
            f"**************************************************\n"
        )
    elif scenario == "sensor_malfunction":
        message_template = (
            f"Commander, Robot {robot_number} has experienced a sensor malfunction, affecting its navigation accuracy. "
            f"Current position: {A}.\n"
            f"**************************************************\n"
        )
    else:
        time_difference = Y - X
        position_difference = (B[0] - A[0], B[1] - A[1])

        message_template = "-" * 50 + f"\nCommander,\n\nRobot {robot_number}'s mission status update:\n"

        # Time-related message
        if time_difference > 0:
            time_message = (
                f"- Estimated Time of Completion: {X} seconds\n"
                f"- Actual Time of Completion: {Y} seconds\n"
                f"- Expected Coordinates: {A}\n"
                f"- Actual Coordinates: {B}\n\n"
                f"The robot is currently experiencing a delay of {time_difference} seconds. \n"
            )
        elif time_difference < 0:
            time_message = (
                f"- Estimated Time of Completion: {X} seconds\n"
                f"- Actual Time of Completion: {Y} seconds\n"
                f"- Expected Coordinates: {A}\n"
                f"- Actual Coordinates: {B}\n\n"
                f"The robot is ahead of schedule by {abs(time_difference)} seconds. \n"
            )
        else:
            time_message = (
                f"- Estimated Time of Completion: {X} seconds\n"
                f"- Actual Time of Completion: {Y} seconds\n"
                f"- Expected Coordinates: {A}\n"
                f"- Actual Coordinates: {B}\n\n"
                f"The robot is on schedule. \n"
            )

        # Position-related message
        position_message = f"It is now located at position {B}."
        if any(position_difference):
            position_message += f" It has moved {abs(position_difference[0])} units " \
                               f"to the {'right' if position_difference[0] > 0 else 'left'}"
            position_message += f" and {abs(position_difference[1])} units " \
                               f"{'upward' if position_difference[1] > 0 else 'downward'}" \
                               f" from the expected position ({A})."

        message_template += time_message + position_message + "\n\n" + "-" * 50

    return message_template


def generate_communication_message(deviation, threshold, hmm_pos1, hmm_pos2, rmm_pos1, rmm_pos2, hmm_time, rmm_time):
    messages = []
    if abs(deviation) > threshold:
        robot_number = 1  # Assuming robot number 1 for example
        X = hmm_time
        Y = rmm_time
        A = (hmm_pos1, hmm_pos2)
        B = (rmm_pos1, rmm_pos2)
        delay_message = generate_delay_message(robot_number, X, Y, A, B)
        # print(delay_message)
        messages.append(delay_message)  # Append formatted delay message to messages list
        # print(messages)
    return messages






def dynamic_deviation_threshold_multi_logic(hmm_arrays, rmm_arrays, update_logic_functions, uncertainty_factor_pos, uncertainty_factor_time):
    all_updated_hmm_arrays = []
    all_communication_messages = []  # Initialize list to collect all communication messages

    for i, update_logic in enumerate(update_logic_functions):
        pos_deviation = calculate_l1_norm(hmm_arrays[i][1:3], rmm_arrays[i][1:3])
        time_deviation = calculate_l1_norm([hmm_arrays[i][3]], [rmm_arrays[i][3]])[0]
        mission_time_deviation = calculate_l1_norm([hmm_arrays[i][4]], [rmm_arrays[i][4]])[0]

        dynamic_threshold_pos = max([max(x, 0.01) for x in pos_deviation])
        dynamic_threshold_time = max(time_deviation, 0.1)


        # dynamic_threshold_mission_time = 10 # 11 stars # dataset low threshold high sensitivity 3
        # dynamic_threshold_mission_time = 15 # 8 stars # dataset medium 2
        dynamic_threshold_mission_time = 20 # 6 stars # dataset high threshold low sensitivity 1


        # dynamic_threshold_mission_time = 40 # 8 stars # 20 datasize jiming


        updated_hmm_array = []
        communication_messages = []  # Initialize list to collect communication messages for this logic iteration

        current_hmm_mission_time = rmm_arrays[0][4]

        for j in range(len(hmm_arrays)):
            updated_pos1 = rmm_arrays[j][1]
            updated_pos2 = rmm_arrays[j][2]
            mission_time_deviation = calculate_l1_norm([current_hmm_mission_time], [rmm_arrays[j][4]])[0]

            hmm_pos1 = hmm_arrays[j][1]
            hmm_pos2 = hmm_arrays[j][2]
            rmm_pos1 = rmm_arrays[j][1]
            rmm_pos2 = rmm_arrays[j][2]
            hmm_time = hmm_arrays[j][3]
            rmm_time = rmm_arrays[j][3]

            messages = generate_communication_message(mission_time_deviation, dynamic_threshold_mission_time,
                                                     hmm_pos1, hmm_pos2, rmm_pos1, rmm_pos2, hmm_time, rmm_time)
            communication_messages.extend(messages)

            updated_mission_time = update_logic(current_hmm_mission_time, mission_time_deviation,
                                               dynamic_threshold_mission_time, uncertainty_factor_time)
            current_hmm_mission_time = updated_mission_time

            updated_hmm_array.append([j, int(round(updated_pos1, 0)), int(round(updated_pos2, 0)), None, updated_mission_time])

        all_updated_hmm_arrays.append(np.array(updated_hmm_array))  # Append updated hmm_array to list
        all_communication_messages.append(communication_messages)  # Append communication messages to list for this logic

    return all_updated_hmm_arrays, all_communication_messages  # Return both arrays and messages

def save_to_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)



if __name__ == "__main__":
    update_logics = [bayesian_probabilistic_update_general]

    uncertainty_factor_pos = 0.05
    uncertainty_factor_time = 0.01

    shmm_data = []

    # Read data from CSV file and format it
    # with open("robot_data_20_jiming.csv", 'r') as csvfile:
    with open("robot_data_10_jiming.csv", 'r') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter='\t')
        timestep = 0
        for row in csv_reader:
            first_state = int(row[1])
            second_state = int(row[2])
            time_elapsed = float(row[3])
            steps_remaining = int(row[4])
            mission_time = timestep + steps_remaining
            shmm_data.append(((first_state, second_state), time_elapsed, mission_time))
            timestep += 1

    rmm_arrays = generate_rmm_array(shmm_data)
    shmm_data_with_index = [[i, pose[0][0], pose[0][1], pose[1], pose[2]] for i, pose in enumerate(shmm_data)]
    hmm_arrays = [[arr[0], arr[1], arr[2], round(arr[3], 2), 37] for arr in shmm_data_with_index]

    all_updated_arrays, communication_details = dynamic_deviation_threshold_multi_logic(
        hmm_arrays, rmm_arrays, update_logics, uncertainty_factor_pos, uncertainty_factor_time
    )

    # print(all_updated_arrays)

    for i, updated_hmm_array in enumerate(all_updated_arrays):
        update_points = []
        last_point = None
        for arr in updated_hmm_array:
            point_of_interest = arr[4]
            if last_point is None or point_of_interest != last_point:
                last_point = point_of_interest
                update_points.append((arr[0], point_of_interest))

        print(f"\nUpdated HMM_array for Update Logic {i + 1} - {update_logics[i].__name__}:")


        # update_points = [(0, 37), (9, 47), (18, 53), (24, 61), (36, 71)]

        filtered_points = [(x, y) for x, y in update_points if x != 0]

        # print(f"filtered Update Points: {filtered_points}")

        print(f"SHMM array: {hmm_arrays}")
        print(f"Updated SHMM array: {np.array(updated_hmm_array)}")
        print(f"RMM Array:{rmm_arrays}")
        print(f"Update Points: {update_points}")
        print(f"Communication Details: {communication_details}")

        # save_to_csv(f"updated_shmm_array_{i+1}.csv", updated_hmm_array, headers=["Index", "Position1", "Position2", "Unused_Time_Taken", "Overall Mission Time"])
        # save_to_csv(f"rmm_array{i+1}.csv", rmm_arrays, headers=["Index", "Position1", "Position2", "Time_Taken", "Overall Mission Time"])
        # save_to_csv(f"update_points_{i+1}.csv", update_points, headers=["Pos_X", "Pos_Y"])





        # save_to_csv(f'SHMM_Base_array_low.csv', hmm_arrays)
        # save_to_csv(f'Updated_SHMM_array_low.csv', np.array(updated_hmm_array).tolist())
        # save_to_csv(f'RMM_array_low.csv', [list(map(int, arr)) for arr in rmm_arrays])
        # save_to_csv(f'Update_points_low.csv', [(int(x), int(y)) for x, y in update_points])



        # save_to_csv(f'SHMM_Base_array_medium.csv', hmm_arrays)
        # save_to_csv(f'Updated_SHMM_array_medium.csv', np.array(updated_hmm_array).tolist())
        # save_to_csv(f'RMM_array_medium.csv', [list(map(int, arr)) for arr in rmm_arrays])
        # save_to_csv(f'Update_points_medium.csv', [(int(x), int(y)) for x, y in update_points])




        save_to_csv(f'SHMM_Base_array_high.csv', hmm_arrays)
        save_to_csv(f'Updated_SHMM_array_high.csv', np.array(updated_hmm_array).tolist())
        save_to_csv(f'RMM_array_high.csv', [list(map(int, arr)) for arr in rmm_arrays])
        save_to_csv(f'Update_points_high.csv', [(int(x), int(y)) for x, y in update_points])



        # Prepare all_messages with 'Mission Starting Point' as the first message
        all_messages = ['Mission Starting Point']

        for sublist in communication_details:
            for message in sublist:
                all_messages.append(message)

        # Save communication details including 'Mission Starting Point'
        # with open(f'communication_details_low.csv', 'w', newline='') as csvfile:
        # with open(f'communication_details_medium.csv', 'w', newline='') as csvfile:
        with open(f'communication_details_high.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Message'])
            for message in all_messages:
                csv_writer.writerow([message])

        # plot_hmm_arrays(
        #     np.array(updated_hmm_array),
        #     rmm_arrays,
        #     update_points=update_points,
        #     # update_points=filtered_points,
        #     communication_details=communication_details
        # )




In [ ]:
import pandas as pd

# Define the path to your input CSV file
# input_file_path = 'robot_data_10 1.csv'
input_file_path = 'robot_data_10 1.csv'
# output_file_path = 'robot_data_10_jiming.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data
data1 = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})

# # Save the DataFrame to a file without headers
# data1.to_csv(output_file_path, index=False, header=False, sep='\t')

# print("Data has been successfully transformed and saved to 'robot_data_10_jiming.csv'.")



df

In [ ]:
data1

In [ ]:

def generate_rmm_array(shmm_data):
    num_tasks = len(shmm_data)

    position_values = [pose[0] for pose in shmm_data]
    time_values = [pose[1] for pose in shmm_data]
    mission_tim_values = [pose[2] for pose in shmm_data]

    # Increase noise standard deviation
    position_noise = np.random.normal(0, 1, num_tasks)  # Adjusted standard deviation
    time_noise = np.random.normal(0, 3, num_tasks)

    rmm_array = []
    prev_time = 0  # Variable to track the previous time value

    for i in range(num_tasks):
        position_value1 = int(np.round(np.abs(position_values[i][0] + position_noise[i])))
        position_value2 = int(np.round(np.abs(position_values[i][1] + np.random.normal(0, 0.1))))
        time_taken = round((np.abs(time_values[i] + time_noise[i])),2)
        mission_tim = int(mission_tim_values[i])

        # Clip position values to be within specified limits
        position_value1 = np.clip(position_value1, 0, 5)
        position_value2 = np.clip(position_value2, 0, 3)

        rmm_array.append([i, position_value1, position_value2, time_taken, mission_tim])

        # Update the previous time value
        prev_time = time_taken + 1  # Increase time for the next iteration

    return rmm_array


import pandas as pd

# Define the path to your input CSV file
# input_file_path = 'robot_data_10 1.csv'
input_file_path = 'robot_data_10 1.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data
data1 = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})






    shmm_data = []

    # Read data from CSV file and format it
    # with open("robot_data_20_jiming.csv", 'r') as csvfile:
    with open("robot_data_10_jiming.csv", 'r') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter='\t')
        timestep = 0
        for row in csv_reader:
            first_state = int(row[1])
            second_state = int(row[2])
            time_elapsed = float(row[3])
            steps_remaining = int(row[4])
            mission_time = timestep + steps_remaining
            shmm_data.append(((first_state, second_state), time_elapsed, mission_time))
            timestep += 1

    rmm_arrays = generate_rmm_array(shmm_data)

In [ ]:
import numpy as np
import pandas as pd

def generate_rmm_array(shmm_data):
    num_tasks = len(shmm_data)

    position_values = [pose[0] for pose in shmm_data]
    time_values = [pose[1] for pose in shmm_data]
    mission_tim_values = [pose[2] for pose in shmm_data]

    # Increase noise standard deviation
    position_noise = np.random.normal(0, 1, num_tasks)  # Adjusted standard deviation
    time_noise = np.random.normal(0, 3, num_tasks)

    rmm_array = []
    prev_time = 0  # Variable to track the previous time value

    for i in range(num_tasks):
        position_value1 = int(np.round(np.abs(position_values[i][0] + position_noise[i])))
        position_value2 = int(np.round(np.abs(position_values[i][1] + np.random.normal(0, 0.1))))
        time_taken = round((np.abs(time_values[i] + time_noise[i])), 2)
        mission_tim = int(mission_tim_values[i])

        # Clip position values to be within specified limits
        position_value1 = np.clip(position_value1, 0, 5)
        position_value2 = np.clip(position_value2, 0, 3)

        rmm_array.append([i, position_value1, position_value2, time_taken, mission_tim])

        # Update the previous time value
        prev_time = time_taken + 1  # Increase time for the next iteration

    return rmm_array

# Define the path to your input CSV file
input_file_path = 'robot_data_10 1.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data and rename it to SHMM
SHMM = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})

# Prepare SHMM data for RMM generation
shmm_data = []
for i, row in SHMM.iterrows():
    first_state = row['Position X']
    second_state = row['Position Y']
    time_elapsed = row['Time Taken']
    steps_remaining = row['Steps Remaining']
    mission_time = i + steps_remaining
    shmm_data.append(((first_state, second_state), time_elapsed, mission_time))

# Generate RMM using SHMM data
RMM_df = generate_rmm_array(shmm_data)

# Convert RMM to DataFrame if needed
RMM = pd.DataFrame(RMM_df, columns=['Task', 'Position X', 'Position Y', 'Time Taken', 'Mission Time'])

shmm_data_with_index = [[i, pose[0][0], pose[0][1], pose[1], pose[2]] for i, pose in enumerate(shmm_data)]
hmm_arrays = [[arr[0], arr[1], arr[2], round(arr[3], 2), 37] for arr in shmm_data_with_index]
hmm_arrays

# Display the RMM and SHMM DataFrames
# print("SHMM:")
# print(SHMM)
# print("\nRMM:")
# print(RMM)


In [ ]:
shmm_data_with_index = [[i, pose[0][0], pose[0][1], pose[1], pose[2]] for i, pose in enumerate(shmm_data)]
hmm_arrays = [[arr[0], arr[1], arr[2], round(arr[3], 2), 37] for arr in shmm_data_with_index]
hmm_arrays

In [1]:
import numpy as np
import pandas as pd

def generate_rmm_array(shmm_data):
    num_tasks = len(shmm_data)

    position_values = [pose[0] for pose in shmm_data]
    time_values = [pose[1] for pose in shmm_data]
    mission_tim_values = [pose[2] for pose in shmm_data]

    # Increase noise standard deviation
    position_noise = np.random.normal(0, 1, num_tasks)  # Adjusted standard deviation
    time_noise = np.random.normal(0, 3, num_tasks)

    rmm_array = []
    prev_time = 0  # Variable to track the previous time value

    for i in range(num_tasks):
        position_value1 = int(np.round(np.abs(position_values[i][0] + position_noise[i])))
        position_value2 = int(np.round(np.abs(position_values[i][1] + np.random.normal(0, 0.1))))
        time_taken = round((np.abs(time_values[i] + time_noise[i])), 2)
        mission_tim = int(mission_tim_values[i])

        # Clip position values to be within specified limits
        position_value1 = np.clip(position_value1, 0, 5)
        position_value2 = np.clip(position_value2, 0, 3)

        rmm_array.append([i, float(position_value1), float(position_value2), time_taken, float(mission_tim)])

        # Update the previous time value
        prev_time = time_taken + 1  # Increase time for the next iteration

    return rmm_array

# Define the path to your input CSV file
input_file_path = 'robot_data_10 1.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data and rename it to SHMM
SHMM = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})

# Prepare SHMM data for RMM generation
shmm_data = []
for i, row in SHMM.iterrows():
    first_state = row['Position X']
    second_state = row['Position Y']
    time_elapsed = row['Time Taken']
    steps_remaining = row['Steps Remaining']
    mission_time = i + steps_remaining
    shmm_data.append(((first_state, second_state), time_elapsed, mission_time))

# Generate RMM using SHMM data
RMM_arrays = generate_rmm_array(shmm_data)

# Prepare hmm_arrays in the desired format
HMM_arrays = [[i, pose[0][0], pose[0][1], round(pose[1], 2), pose[2]] for i, pose in enumerate(shmm_data)]

# Print the arrays without headers
print("HMM Arrays:")
for row in HMM_arrays:
    print(row)

print("\nRMM Arrays:")
for row in RMM_arrays:
    print(row)


HMM Arrays:
[0, 4.0, 3.0, 10.0, 48.0]
[1, 4.0, 2.0, 20.0, 48.0]
[2, 4.0, 1.0, 30.0, 48.0]
[3, 3.0, 1.0, 40.0, 52.0]
[4, 2.0, 1.0, 50.0, 56.0]
[5, 1.0, 1.0, 60.0, 60.0]
[6, 1.0, 0.0, 110.0, 62.0]
[7, 2.0, 0.0, 120.0, 62.0]
[8, 3.0, 0.0, 130.0, 62.0]
[9, 4.0, 0.0, 140.0, 62.0]
[10, 5.0, 0.0, 150.0, 62.0]
[11, 6.0, 0.0, 160.0, 66.0]
[12, 7.0, 0.0, 170.0, 70.0]
[13, 8.0, 0.0, 180.0, 74.0]
[14, 9.0, 0.0, 190.0, 78.0]
[15, 9.0, 1.0, 200.0, 78.0]
[16, 9.0, 2.0, 210.0, 78.0]
[17, 9.0, 3.0, 220.0, 78.0]
[18, 9.0, 4.0, 230.0, 78.0]
[19, 9.0, 3.0, 240.0, 88.0]
[20, 9.0, 2.0, 250.0, 88.0]
[21, 9.0, 1.0, 260.0, 88.0]
[22, 8.0, 1.0, 310.0, 88.0]
[23, 7.0, 1.0, 320.0, 88.0]
[24, 6.0, 1.0, 330.0, 88.0]
[25, 5.0, 1.0, 340.0, 88.0]
[26, 5.0, 2.0, 350.0, 88.0]
[27, 5.0, 3.0, 360.0, 88.0]
[28, 5.0, 4.0, 370.0, 88.0]
[29, 5.0, 3.0, 380.0, 88.0]
[30, 5.0, 2.0, 390.0, 88.0]
[31, 5.0, 1.0, 400.0, 88.0]
[32, 6.0, 1.0, 410.0, 88.0]
[33, 7.0, 1.0, 420.0, 88.0]
[34, 8.0, 1.0, 430.0, 88.0]
[35, 9.0, 1.0, 440.0, 88

In [27]:
import numpy as np
import pandas as pd

def generate_rmm_array(shmm_data):
    num_tasks = len(shmm_data)

    position_values = [pose[0] for pose in shmm_data]
    time_values = [pose[1] for pose in shmm_data]
    mission_tim_values = [pose[2] for pose in shmm_data]

    # Increase noise standard deviation
    position_noise = 0 # Adjusted standard deviation
    time_noise = np.random.normal(0, 3, num_tasks)

    rmm_array = []
    prev_time = 0  # Variable to track the previous time value

    for i in range(num_tasks):
        position_value1 = int(np.round(np.abs(position_values[i][0] + position_noise)))
        position_value2 = int(np.round(np.abs(position_values[i][1] + np.random.normal(0, 0.1))))
        time_taken = round((np.abs(time_values[i] + time_noise[i])), 2)
        mission_tim = int(mission_tim_values[i])

        # # Clip position values to be within specified limits
        # position_value1 = np.clip(position_value1, 0, 5)
        # position_value2 = np.clip(position_value2, 0, 3)

        rmm_array.append([i, float(position_value1), float(position_value2), time_taken, float(mission_tim)])

        # Update the previous time value
        prev_time = time_taken + 1  # Increase time for the next iteration

    return rmm_array


# import numpy as np
# import pandas as pd

# def generate_rmm_array(shmm_data):
#     num_tasks = len(shmm_data)

#     position_values = [pose[0] for pose in shmm_data]
#     time_values = [pose[1] for pose in shmm_data]
#     mission_tim_values = [pose[2] for pose in shmm_data]

#     # Increase noise standard deviation
#     position_noise = np.random.normal(0, 1, num_tasks)  # Adjusted standard deviation
#     time_noise = np.random.normal(0, 3, num_tasks)

#     rmm_array = []

#     # Start from the first position
#     current_position1 = position_values[0][0]
#     current_position2 = position_values[0][1]

#     for i in range(num_tasks):
#         time_taken = round((np.abs(time_values[i] + time_noise[i])), 2)
#         mission_tim = int(mission_tim_values[i])

#         # Move one unit in either X or Y direction
#         if np.random.rand() < 0.5:  # Randomly choose to move in X direction
#             current_position1 += np.random.choice([-1, 1])  # Move left or right
#         else:  # Move in Y direction
#             current_position2 += np.random.choice([-1, 1])  # Move up or down

#         # # Clip position values to be within specified limits
#         # current_position1 = np.clip(current_position1, 0, 5)
#         # current_position2 = np.clip(current_position2, 0, 3)

#         rmm_array.append([i, float(current_position1), float(current_position2), time_taken, float(mission_tim)])

#     return rmm_array

# Example usage:
# Generate sample SHMM data (the same way you defined before)
# Generate RMM using SHMM data
RMM_arrays = generate_rmm_array(shmm_data)

# Prepare hmm_arrays in the desired format
HMM_arrays = [[i, pose[0][0], pose[0][1], round(pose[1], 2), pose[2]] for i, pose in enumerate(shmm_data)]

# Print the arrays without headers
print("HMM Arrays:")
for row in HMM_arrays:
    print(row)

print("\nRMM Arrays:")
for row in RMM_arrays:
    print(row)


# Define the path to your input CSV file
input_file_path = 'robot_data_10 1.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data and rename it to SHMM
SHMM = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})

# Prepare SHMM data for RMM generation
shmm_data = []
for i, row in SHMM.iterrows():
    first_state = row['Position X']
    second_state = row['Position Y']
    time_elapsed = row['Time Taken']
    steps_remaining = row['Steps Remaining']
    mission_time = i + steps_remaining
    shmm_data.append(((first_state, second_state), time_elapsed, mission_time))

# Generate RMM using SHMM data
RMM_arrays = generate_rmm_array(shmm_data)

# Prepare hmm_arrays in the desired format
HMM_arrays = [[i, pose[0][0], pose[0][1], round(pose[1], 2), pose[2]] for i, pose in enumerate(shmm_data)]

# Print the arrays without headers
print("HMM Arrays:")
for row in HMM_arrays:
    print(row)

print("\nRMM Arrays:")
for row in RMM_arrays:
    print(row)


HMM Arrays:
[0, 4.0, 3.0, 10.0, 48.0]
[1, 4.0, 2.0, 20.0, 48.0]
[2, 4.0, 1.0, 30.0, 48.0]
[3, 3.0, 1.0, 40.0, 52.0]
[4, 2.0, 1.0, 50.0, 56.0]
[5, 1.0, 1.0, 60.0, 60.0]
[6, 1.0, 0.0, 110.0, 62.0]
[7, 2.0, 0.0, 120.0, 62.0]
[8, 3.0, 0.0, 130.0, 62.0]
[9, 4.0, 0.0, 140.0, 62.0]
[10, 5.0, 0.0, 150.0, 62.0]
[11, 6.0, 0.0, 160.0, 66.0]
[12, 7.0, 0.0, 170.0, 70.0]
[13, 8.0, 0.0, 180.0, 74.0]
[14, 9.0, 0.0, 190.0, 78.0]
[15, 9.0, 1.0, 200.0, 78.0]
[16, 9.0, 2.0, 210.0, 78.0]
[17, 9.0, 3.0, 220.0, 78.0]
[18, 9.0, 4.0, 230.0, 78.0]
[19, 9.0, 3.0, 240.0, 88.0]
[20, 9.0, 2.0, 250.0, 88.0]
[21, 9.0, 1.0, 260.0, 88.0]
[22, 8.0, 1.0, 310.0, 88.0]
[23, 7.0, 1.0, 320.0, 88.0]
[24, 6.0, 1.0, 330.0, 88.0]
[25, 5.0, 1.0, 340.0, 88.0]
[26, 5.0, 2.0, 350.0, 88.0]
[27, 5.0, 3.0, 360.0, 88.0]
[28, 5.0, 4.0, 370.0, 88.0]
[29, 5.0, 3.0, 380.0, 88.0]
[30, 5.0, 2.0, 390.0, 88.0]
[31, 5.0, 1.0, 400.0, 88.0]
[32, 6.0, 1.0, 410.0, 88.0]
[33, 7.0, 1.0, 420.0, 88.0]
[34, 8.0, 1.0, 430.0, 88.0]
[35, 9.0, 1.0, 440.0, 88

In [28]:
RMM_arrays

[[0, 4.0, 3.0, 9.08, 48.0],
 [1, 4.0, 2.0, 12.44, 48.0],
 [2, 4.0, 1.0, 28.74, 48.0],
 [3, 3.0, 1.0, 43.95, 52.0],
 [4, 2.0, 1.0, 49.98, 56.0],
 [5, 1.0, 1.0, 61.31, 60.0],
 [6, 1.0, 0.0, 117.1, 62.0],
 [7, 2.0, 0.0, 120.73, 62.0],
 [8, 3.0, 0.0, 132.1, 62.0],
 [9, 4.0, 0.0, 139.81, 62.0],
 [10, 5.0, 0.0, 151.51, 62.0],
 [11, 6.0, 0.0, 159.16, 66.0],
 [12, 7.0, 0.0, 168.56, 70.0],
 [13, 8.0, 0.0, 178.71, 74.0],
 [14, 9.0, 0.0, 189.69, 78.0],
 [15, 9.0, 1.0, 198.56, 78.0],
 [16, 9.0, 2.0, 206.38, 78.0],
 [17, 9.0, 3.0, 211.38, 78.0],
 [18, 9.0, 4.0, 230.95, 78.0],
 [19, 9.0, 3.0, 237.48, 88.0],
 [20, 9.0, 2.0, 249.69, 88.0],
 [21, 9.0, 1.0, 261.46, 88.0],
 [22, 8.0, 1.0, 312.23, 88.0],
 [23, 7.0, 1.0, 322.07, 88.0],
 [24, 6.0, 1.0, 329.21, 88.0],
 [25, 5.0, 1.0, 342.25, 88.0],
 [26, 5.0, 2.0, 351.36, 88.0],
 [27, 5.0, 3.0, 361.99, 88.0],
 [28, 5.0, 4.0, 371.52, 88.0],
 [29, 5.0, 3.0, 380.69, 88.0],
 [30, 5.0, 2.0, 389.04, 88.0],
 [31, 5.0, 1.0, 397.96, 88.0],
 [32, 6.0, 1.0, 410.33, 88.

In [8]:
# RMM_arrays[i:3]

In [7]:
# # Read the input CSV file to get steps remaining from df

# # Reformat RMM_arrays into the desired format
# reformatted_rmm = []
# for i, row in enumerate(RMM_arrays):
#     position = (int(row[1]), int(row[2]))
#     time_taken = row[3]
#     steps_remaining = int(df.iloc[i, 2])  # Borrowing the third column from df

#     reformatted_rmm.append([position, time_taken, steps_remaining, int(row[4])])

# # Convert to DataFrame and print
# RMM_df = pd.DataFrame(reformatted_rmm, columns=[0, 1, 2, 3])
# # print(RMM_df)
# RMM_df

In [29]:
# Read the last column from RMM_arrays separately
last_column = [row[-1] for row in RMM_arrays]

# Reformat RMM_arrays into the desired format
reformatted_rmm = []
for i, row in enumerate(RMM_arrays):
    position = (int(row[1]), int(row[2]))  # Position as (X, Y)
    time_taken = round(row[3], 2)          # Rounded time taken
    last_value = last_column[i]            # Last column value from RMM_arrays
    
    reformatted_rmm.append([position, time_taken, last_value])  # Append to reformatted list

# Convert to DataFrame and print
RMM_df = pd.DataFrame(reformatted_rmm, columns=[0, 1, 2])
RMM_df.to_csv('RMM.csv', index=False, header=False)
RMM_df


,0,1,2
0,"(4, 3)",9.08,48.0
1,"(4, 2)",12.44,48.0
2,"(4, 1)",28.74,48.0
3,"(3, 1)",43.95,52.0
4,"(2, 1)",49.98,56.0
...,...,...,...
212,"(5, 4)",2442.35,216.0
213,"(5, 3)",2451.77,216.0
214,"(5, 3)",2457.63,216.0
215,"(5, 2)",2457.27,216.0


In [11]:
# # Read the input CSV file
# df = pd.read_csv('robot_data_10 1.csv', header=None)

# # Reformat RMM_arrays into the desired format with the last column from df
# reformatted_rmm = []
# for i, row in enumerate(RMM_arrays):
#     position = (int(row[1]), int(row[2]))
#     time_taken = row[3]
#     steps_remaining = int(df.iloc[i, 2])  # Borrowing the third column from df
#     mission_time = int(df.iloc[i, 4])  # Borrowing the last column from df

#     reformatted_rmm.append([position, time_taken, steps_remaining, mission_time])

# # Convert to DataFrame
# RMM_df = pd.DataFrame(reformatted_rmm, columns=[0, 1, 2, 3])

# # Save as CSV
# # Save RMM_df as RMM.csv without column headers
# # RMM_df.to_csv('RMM.csv', index=False, header=False)

# RMM_df


In [30]:
# input_file_path = 'robot_data_10 1.csv'
# output_file_path = 'robot_data_10_jiming.csv'

# Read the input CSV file
df_og = pd.read_csv('robot_data_10 1.csv', header=None)
df_og

,0,1,2,3
0,"(4, 3)",10,48,470
1,"(4, 2)",20,47,460
2,"(4, 1)",30,46,450
3,"(3, 1)",40,49,480
4,"(2, 1)",50,52,510
...,...,...,...,...
212,"(5, 4)",2440,4,40
213,"(5, 3)",2450,3,30
214,"(5, 3)",2450,2,20
215,"(5, 2)",2460,1,10


In [31]:
df_ng = pd.read_csv('RMM.csv',header=None)
df_ng

,0,1,2
0,"(4, 3)",9.08,48.0
1,"(4, 2)",12.44,48.0
2,"(4, 1)",28.74,48.0
3,"(3, 1)",43.95,52.0
4,"(2, 1)",49.98,56.0
...,...,...,...
212,"(5, 4)",2442.35,216.0
213,"(5, 3)",2451.77,216.0
214,"(5, 3)",2457.63,216.0
215,"(5, 2)",2457.27,216.0


In [4]:
HMM_arrays

[[0, 4.0, 3.0, 10.0, 48.0],
 [1, 4.0, 2.0, 20.0, 48.0],
 [2, 4.0, 1.0, 30.0, 48.0],
 [3, 3.0, 1.0, 40.0, 52.0],
 [4, 2.0, 1.0, 50.0, 56.0],
 [5, 1.0, 1.0, 60.0, 60.0],
 [6, 1.0, 0.0, 110.0, 62.0],
 [7, 2.0, 0.0, 120.0, 62.0],
 [8, 3.0, 0.0, 130.0, 62.0],
 [9, 4.0, 0.0, 140.0, 62.0],
 [10, 5.0, 0.0, 150.0, 62.0],
 [11, 6.0, 0.0, 160.0, 66.0],
 [12, 7.0, 0.0, 170.0, 70.0],
 [13, 8.0, 0.0, 180.0, 74.0],
 [14, 9.0, 0.0, 190.0, 78.0],
 [15, 9.0, 1.0, 200.0, 78.0],
 [16, 9.0, 2.0, 210.0, 78.0],
 [17, 9.0, 3.0, 220.0, 78.0],
 [18, 9.0, 4.0, 230.0, 78.0],
 [19, 9.0, 3.0, 240.0, 88.0],
 [20, 9.0, 2.0, 250.0, 88.0],
 [21, 9.0, 1.0, 260.0, 88.0],
 [22, 8.0, 1.0, 310.0, 88.0],
 [23, 7.0, 1.0, 320.0, 88.0],
 [24, 6.0, 1.0, 330.0, 88.0],
 [25, 5.0, 1.0, 340.0, 88.0],
 [26, 5.0, 2.0, 350.0, 88.0],
 [27, 5.0, 3.0, 360.0, 88.0],
 [28, 5.0, 4.0, 370.0, 88.0],
 [29, 5.0, 3.0, 380.0, 88.0],
 [30, 5.0, 2.0, 390.0, 88.0],
 [31, 5.0, 1.0, 400.0, 88.0],
 [32, 6.0, 1.0, 410.0, 88.0],
 [33, 7.0, 1.0, 420.0, 88.

In [2]:
RMM_arrays

[[0, 3.0, 3.0, 12.91, 48.0],
 [1, 4.0, 2.0, 22.15, 48.0],
 [2, 5.0, 1.0, 33.92, 48.0],
 [3, 3.0, 1.0, 42.68, 52.0],
 [4, 2.0, 1.0, 51.01, 56.0],
 [5, 1.0, 1.0, 62.99, 60.0],
 [6, 1.0, 0.0, 110.07, 62.0],
 [7, 1.0, 0.0, 115.2, 62.0],
 [8, 4.0, 0.0, 126.41, 62.0],
 [9, 3.0, 0.0, 138.75, 62.0],
 [10, 5.0, 0.0, 156.1, 62.0],
 [11, 5.0, 0.0, 160.89, 66.0],
 [12, 5.0, 0.0, 169.05, 70.0],
 [13, 5.0, 0.0, 179.58, 74.0],
 [14, 5.0, 0.0, 189.39, 78.0],
 [15, 5.0, 1.0, 199.39, 78.0],
 [16, 5.0, 2.0, 207.79, 78.0],
 [17, 5.0, 3.0, 215.7, 78.0],
 [18, 5.0, 3.0, 226.22, 78.0],
 [19, 5.0, 3.0, 235.76, 88.0],
 [20, 5.0, 2.0, 251.85, 88.0],
 [21, 5.0, 1.0, 258.04, 88.0],
 [22, 5.0, 1.0, 311.51, 88.0],
 [23, 4.0, 1.0, 320.7, 88.0],
 [24, 5.0, 1.0, 327.96, 88.0],
 [25, 4.0, 1.0, 341.14, 88.0],
 [26, 4.0, 2.0, 350.84, 88.0],
 [27, 4.0, 3.0, 361.69, 88.0],
 [28, 5.0, 3.0, 374.38, 88.0],
 [29, 5.0, 3.0, 376.07, 88.0],
 [30, 5.0, 2.0, 390.48, 88.0],
 [31, 5.0, 1.0, 399.8, 88.0],
 [32, 5.0, 1.0, 411.52, 88.0]

In [ ]:

# Define the path to your input CSV file
input_file_path = 'robot_data_10 1.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path, header=None)

# Initialize lists to store the parsed data
index = []
position_x = []
position_y = []
time_taken = []
steps_remaining = []

# Parse the data
for i, row in df.iterrows():
    pos = eval(row[0])  # Evaluate the position tuple (x, y)
    index.append(i + 1)  # Create index starting from 1
    position_x.append(pos[0])
    position_y.append(pos[1])
    time_taken.append(float(row[1]))
    steps_remaining.append(int(row[2]))

# Create a DataFrame with the parsed data and rename it to SHMM
SHMM = pd.DataFrame({
    'Index': index,
    'Position X': position_x,
    'Position Y': position_y,
    'Time Taken': time_taken,
    'Steps Remaining': steps_remaining
})

# Prepare SHMM data for RMM generation
shmm_data = []
for i, row in SHMM.iterrows():
    first_state = row['Position X']
    second_state = row['Position Y']
    time_elapsed = row['Time Taken']
    steps_remaining = row['Steps Remaining']
    mission_time = i + steps_remaining
    shmm_data.append(((first_state, second_state), time_elapsed, mission_time))


# Prepare hmm_arrays in the desired format
HMM_arrays = [[i, pose[0][0], pose[0][1], round(pose[1], 2), pose[2]] for i, pose in enumerate(shmm_data)]





In [20]:
import pandas as pd

def generate_hmm_arrays(input_file_path):
    # Read the input CSV file
    df = pd.read_csv(input_file_path, header=None)

    # Initialize lists to store the parsed data
    index = []
    position_x = []
    position_y = []
    time_taken = []
    steps_remaining = []

    # Parse the data
    for i, row in df.iterrows():
        pos = eval(row[0])  # Evaluate the position tuple (x, y)
        index.append(i + 1)  # Create index starting from 1
        position_x.append(pos[0])
        position_y.append(pos[1])
        time_taken.append(float(row[1]))
        steps_remaining.append(int(row[2]))

    # Create a DataFrame with the parsed data and rename it to SHMM
    SHMM = pd.DataFrame({
        'Index': index,
        'Position X': position_x,
        'Position Y': position_y,
        'Time Taken': time_taken,
        'Steps Remaining': steps_remaining
    })

    # Prepare SHMM data for RMM generation
    shmm_data = []
    for i, row in SHMM.iterrows():
        first_state = row['Position X']
        second_state = row['Position Y']
        time_elapsed = row['Time Taken']
        steps_remaining = row['Steps Remaining']
        mission_time = i + steps_remaining
        shmm_data.append(((first_state, second_state), time_elapsed, mission_time))

    # Prepare hmm_arrays in the desired format
    HMM_arrays = [[i, pose[0][0], pose[0][1], round(pose[1], 2), pose[2]] for i, pose in enumerate(shmm_data)]
    
    return HMM_arrays

def select_hmm_array(hmm_arrays, timestep):
    """
    Select a row of data from HMM_arrays based on the given timestep.

    Parameters:
    hmm_arrays (list): The HMM arrays generated from the input CSV file.
    timestep (int): The index value (1-based) for which to retrieve the corresponding row.

    Returns:
    list: The corresponding row of data from the HMM arrays, or None if not found.
    """
    # Adjust for 0-based index in Python
    index = timestep - 1
    
    # Check if the index is valid
    if 0 <= index < len(hmm_arrays):
        return hmm_arrays[index]
    else:
        return None  # Return None if the index is out of bounds

# Example usage
input_file_path = 'robot_data_10 1.csv'
hmm_arrays = generate_hmm_arrays(input_file_path)



In [21]:
hmm_arrays


[[0, 4.0, 3.0, 10.0, 48.0],
 [1, 4.0, 2.0, 20.0, 48.0],
 [2, 4.0, 1.0, 30.0, 48.0],
 [3, 3.0, 1.0, 40.0, 52.0],
 [4, 2.0, 1.0, 50.0, 56.0],
 [5, 1.0, 1.0, 60.0, 60.0],
 [6, 1.0, 0.0, 110.0, 62.0],
 [7, 2.0, 0.0, 120.0, 62.0],
 [8, 3.0, 0.0, 130.0, 62.0],
 [9, 4.0, 0.0, 140.0, 62.0],
 [10, 5.0, 0.0, 150.0, 62.0],
 [11, 6.0, 0.0, 160.0, 66.0],
 [12, 7.0, 0.0, 170.0, 70.0],
 [13, 8.0, 0.0, 180.0, 74.0],
 [14, 9.0, 0.0, 190.0, 78.0],
 [15, 9.0, 1.0, 200.0, 78.0],
 [16, 9.0, 2.0, 210.0, 78.0],
 [17, 9.0, 3.0, 220.0, 78.0],
 [18, 9.0, 4.0, 230.0, 78.0],
 [19, 9.0, 3.0, 240.0, 88.0],
 [20, 9.0, 2.0, 250.0, 88.0],
 [21, 9.0, 1.0, 260.0, 88.0],
 [22, 8.0, 1.0, 310.0, 88.0],
 [23, 7.0, 1.0, 320.0, 88.0],
 [24, 6.0, 1.0, 330.0, 88.0],
 [25, 5.0, 1.0, 340.0, 88.0],
 [26, 5.0, 2.0, 350.0, 88.0],
 [27, 5.0, 3.0, 360.0, 88.0],
 [28, 5.0, 4.0, 370.0, 88.0],
 [29, 5.0, 3.0, 380.0, 88.0],
 [30, 5.0, 2.0, 390.0, 88.0],
 [31, 5.0, 1.0, 400.0, 88.0],
 [32, 6.0, 1.0, 410.0, 88.0],
 [33, 7.0, 1.0, 420.0, 88.

In [22]:
# Select a row for a given timestep
timestep = 5  # Example timestep
selected_row = select_rmm_array(hmm_arrays, timestep)
selected_row

[4, 2.0, 1.0, 50.0, 56.0]